In [5]:
# 스타벅스 크롤링 분석
from selenium import webdriver
import time
import requests
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

url = 'https://www.starbucks.co.kr/store/store_map.do'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

area_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element(By.CSS_SELECTOR, area_btn).click() 
time.sleep(2)

seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, seoul_btn).click()
time.sleep(2)

all_btn =  '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, all_btn).click()
time.sleep(2)

txt = driver.page_source
html = bs(txt)


C:\Users\두현\AppData\Local\Temp\ipykernel_19164\1533751495.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


In [64]:
# 서울은 총 600개인데 내 주역 지변 스타벅스 까지 같이 나옴
len(html.select('li.quickResultLstCon')) # 610개

shops = html.select('ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon')
shop = shops[0]
shop

600

In [69]:
# 위도
lat = shop['data-lat']
print(lat)
# 경도
lng = shop['data-long']
print(lng)
# 지점명
name = shop['data-name']
print(name)
# 주소
addr = str(shop.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
print(addr)
# 전화번호
tel = str(shop.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
print(tel)
# 매장종류
shoptype = shop.select('i')[0].text
print(shoptype)

37.501087
127.043069
역삼아레나빌딩
서울특별시 강남구 언주로 425 (역삼동)
1522-3232
리저브 매장 2번


In [70]:
starbucks_list = []

for shop in shops:
    lat = shop['data-lat']
    lng = shop['data-long']
    name = shop['data-name']
    addr = str(shop.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
    tel = str(shop.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
    shoptype = shop.select('i')[0].text

    time.sleep(0.1)
    starbucks_list.append([name, lat, lng, addr, tel, shoptype])

import pandas as pd
df = pd.DataFrame(starbucks_list, columns = ['매장명', '위도', '경도', '주소', '전화번호', '매장종류'])
df

,매장명,위도,경도,주소,전화번호,매장종류
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,리저브 매장 2번
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,리저브 매장 2번
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,리저브 매장 2번
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,리저브 매장 2번
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,리저브 매장 2번
...,...,...,...,...,...,...
595,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,리저브 매장 2번
596,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,리저브 매장 2번
597,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,리저브 매장 2번
598,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,리저브 매장 2번


In [74]:
# 엑셀로 만들기
# df.to_excel('starbucks_seoul.xlsx', index = False)
# df = pd.read_excel('starbucks_seoul.xlsx', header = 0)

In [76]:
##############################################################
#                       데이터 분석                           #
##############################################################

# !pip install folium

In [81]:
import folium

starmap = folium.Map(
    location = [37.5666805, 126.9784147], # 구글에서 검색한 서울 시청 좌표
    zoom_start = 11

)

starmap


In [83]:
import folium

starmap = folium.Map(
    location = [37.5666805, 126.9784147], # 구글에서 검색한 서울 시청 좌표
    zoom_start = 11,
    tiles = 'Stamen Terrain'

)

# 타일 종류
# OpenStreetMap (기본값)
# Stamen Terrain, Stamen Toner, Stamen Watercolor
# CartoDB positron, CartoDB dark_matter
starmap

In [92]:
import folium

starmap = folium.Map(
    location = [37.5666805, 126.9784147], # 구글에서 검색한 서울 시청 좌표
    zoom_start = 11,
    tiles = 'Stamen Terrain'

)

# for shop in shops:
#     lat = shop['data-lat']
#     lng = shop['data-long']
#     name = shop['data-name']
#     addr = str(shop.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]
#     tel = str(shop.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]
#     shoptype = shop.select('i')[0].text

#     time.sleep(0.1)
#     starbucks_list.append([name, lat, lng, addr, tel, shoptype])
    
#     folium.Marker(
#     location = [lat, lng],
#     fill = True
# ).add_to(starmap)

for idx in df.index:
    lat = df.loc[idx, '위도']
    lng = df.loc[idx, '경도']

    folium.Marker(
        location = [lat, lng],
        fill = True
    ).add_to(starmap)


starmap

In [100]:
# 각 지점에서 '구'명 추출 

gu_names = []

for addr in df['주소']:
    gu = addr.split()[1]
    gu_names.append(gu)

print(gu_names)    
len(gu_names)

['강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강남구', '강북구', '강북구', '강북구', '강북구', '강북구', '강북구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '강서구', '관악구', '관악구', '관악구', '관악구', '관악구', '관악구', '관악구', '관악구', '관악구', '관악구', '관악구', '관악구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구', '광진구'

600

In [102]:
# 기존의 데이터프레임에서 열을 하나 추가
df['시군구명'] = gu_names
df


,매장명,위도,경도,주소,전화번호,매장종류,시군구명
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,리저브 매장 2번,강남구
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,리저브 매장 2번,강남구
2,신사역성일빌딩,37.513931,127.020606,서울특별시 강남구 강남대로 584 (논현동),1522-3232,리저브 매장 2번,강남구
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,리저브 매장 2번,강남구
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,리저브 매장 2번,강남구
...,...,...,...,...,...,...,...
595,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,리저브 매장 2번,중랑구
596,상봉역,37.596890,127.086470,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,리저브 매장 2번,중랑구
597,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,리저브 매장 2번,중랑구
598,양원역,37.606654,127.106360,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,리저브 매장 2번,중랑구


In [118]:
# '구'별 스타벅스 매장 수 계산

starbucks_count = df.pivot_table(index='시군구명', values='매장명', aggfunc='count').rename(columns={'매장명':'매장수'})
starbucks_count

,매장수
시군구명,
강남구,87
강동구,17
강북구,6
강서구,25
관악구,12
광진구,19
구로구,14
금천구,13
노원구,14


In [119]:

seoul_sgg = pd.read_excel('seoul_sgg_stat.xlsx', thousands=',')
seoul_sgg

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y
0,11320,도봉구,37.665861,127.031767,2,371095,68669,18455,68669,18455
1,11380,은평구,37.617612,126.922700,8,474165,87693,24179,87693,24179
2,11230,동대문구,37.583801,127.050700,8,374039,143858,32994,143858,32994
3,11590,동작구,37.496504,126.944307,11,407966,103915,19609,103915,19609
4,11545,금천구,37.460097,126.900155,11,262337,223058,30080,223058,30080
5,11530,구로구,37.495486,126.858121,11,449187,210506,37445,210506,37445
6,11110,종로구,37.599100,126.986149,37,177023,269106,40871,269106,40871
7,11305,강북구,37.646995,127.014716,5,342669,69787,18654,69787,18654
8,11260,중랑구,37.595379,127.093967,7,431283,99241,27287,99241,27287
9,11680,강남구,37.495985,127.066409,81,570913,711278,73590,711278,73590


In [120]:
# 거주자 수, 업체 수 등의 엑셀 파일과 결합하기
seoul_sgg = pd.merge(seoul_sgg, starbucks_count, how='left', on='시군구명')
seoul_sgg

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y,매장수
0,11320,도봉구,37.665861,127.031767,2,371095,68669,18455,68669,18455,4
1,11380,은평구,37.617612,126.922700,8,474165,87693,24179,87693,24179,13
2,11230,동대문구,37.583801,127.050700,8,374039,143858,32994,143858,32994,10
3,11590,동작구,37.496504,126.944307,11,407966,103915,19609,103915,19609,11
4,11545,금천구,37.460097,126.900155,11,262337,223058,30080,223058,30080,13
5,11530,구로구,37.495486,126.858121,11,449187,210506,37445,210506,37445,14
6,11110,종로구,37.599100,126.986149,37,177023,269106,40871,269106,40871,40
7,11305,강북구,37.646995,127.014716,5,342669,69787,18654,69787,18654,6
8,11260,중랑구,37.595379,127.093967,7,431283,99241,27287,99241,27287,8
9,11680,강남구,37.495985,127.066409,81,570913,711278,73590,711278,73590,87


In [122]:
# 주민등록인구 십만명당 매장수

seoul_sgg['거주자_십만명당_매장수'] = seoul_sgg['매장수'] / (seoul_sgg['주민등록인구']/100000)
seoul_sgg

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y,매장수,거주자_십만명당_매장수
0,11320,도봉구,37.665861,127.031767,2,371095,68669,18455,68669,18455,4,1.077891
1,11380,은평구,37.617612,126.922700,8,474165,87693,24179,87693,24179,13,2.741662
2,11230,동대문구,37.583801,127.050700,8,374039,143858,32994,143858,32994,10,2.673518
3,11590,동작구,37.496504,126.944307,11,407966,103915,19609,103915,19609,11,2.696303
4,11545,금천구,37.460097,126.900155,11,262337,223058,30080,223058,30080,13,4.955458
5,11530,구로구,37.495486,126.858121,11,449187,210506,37445,210506,37445,14,3.116742
6,11110,종로구,37.599100,126.986149,37,177023,269106,40871,269106,40871,40,22.595934
7,11305,강북구,37.646995,127.014716,5,342669,69787,18654,69787,18654,6,1.750961
8,11260,중랑구,37.595379,127.093967,7,431283,99241,27287,99241,27287,8,1.854931
9,11680,강남구,37.495985,127.066409,81,570913,711278,73590,711278,73590,87,15.238749


In [125]:
# 종사자수 십만명당 매장수
seoul_sgg['종사자수_십만명당_매장수'] = seoul_sgg['매장수'] / (seoul_sgg['종사자수_y']/100000)
seoul_sgg

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y,매장수,거주자_십만명당_매장수,종사자수십만명당_매장수,종사자수_십만명당_매장수
0,11320,도봉구,37.665861,127.031767,2,371095,68669,18455,68669,18455,4,1.077891,5.825045,5.825045
1,11380,은평구,37.617612,126.922700,8,474165,87693,24179,87693,24179,13,2.741662,14.824444,14.824444
2,11230,동대문구,37.583801,127.050700,8,374039,143858,32994,143858,32994,10,2.673518,6.951299,6.951299
3,11590,동작구,37.496504,126.944307,11,407966,103915,19609,103915,19609,11,2.696303,10.585575,10.585575
4,11545,금천구,37.460097,126.900155,11,262337,223058,30080,223058,30080,13,4.955458,5.828081,5.828081
5,11530,구로구,37.495486,126.858121,11,449187,210506,37445,210506,37445,14,3.116742,6.650642,6.650642
6,11110,종로구,37.599100,126.986149,37,177023,269106,40871,269106,40871,40,22.595934,14.864031,14.864031
7,11305,강북구,37.646995,127.014716,5,342669,69787,18654,69787,18654,6,1.750961,8.597590,8.597590
8,11260,중랑구,37.595379,127.093967,7,431283,99241,27287,99241,27287,8,1.854931,8.061184,8.061184
9,11680,강남구,37.495985,127.066409,81,570913,711278,73590,711278,73590,87,15.238749,12.231504,12.231504


In [128]:
# 사업체수 10000개당 매장수
seoul_sgg['사업체수_만개당_매장수'] = seoul_sgg['매장수'] / (seoul_sgg['사업체수_y']/10000)
seoul_sgg

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y,매장수,거주자_십만명당_매장수,종사자수십만명당_매장수,종사자수_십만명당_매장수,사업체수_천개당_매장수,사업체수_만개당_매장수
0,11320,도봉구,37.665861,127.031767,2,371095,68669,18455,68669,18455,4,1.077891,5.825045,5.825045,2.167434,2.167434
1,11380,은평구,37.617612,126.922700,8,474165,87693,24179,87693,24179,13,2.741662,14.824444,14.824444,5.376566,5.376566
2,11230,동대문구,37.583801,127.050700,8,374039,143858,32994,143858,32994,10,2.673518,6.951299,6.951299,3.030854,3.030854
3,11590,동작구,37.496504,126.944307,11,407966,103915,19609,103915,19609,11,2.696303,10.585575,10.585575,5.609669,5.609669
4,11545,금천구,37.460097,126.900155,11,262337,223058,30080,223058,30080,13,4.955458,5.828081,5.828081,4.321809,4.321809
5,11530,구로구,37.495486,126.858121,11,449187,210506,37445,210506,37445,14,3.116742,6.650642,6.650642,3.738817,3.738817
6,11110,종로구,37.599100,126.986149,37,177023,269106,40871,269106,40871,40,22.595934,14.864031,14.864031,9.786890,9.786890
7,11305,강북구,37.646995,127.014716,5,342669,69787,18654,69787,18654,6,1.750961,8.597590,8.597590,3.216468,3.216468
8,11260,중랑구,37.595379,127.093967,7,431283,99241,27287,99241,27287,8,1.854931,8.061184,8.061184,2.931799,2.931799
9,11680,강남구,37.495985,127.066409,81,570913,711278,73590,711278,73590,87,15.238749,12.231504,12.231504,11.822258,11.822258
